In [17]:
cd

/Users/megz


In [23]:
import json
import pandas as pd

# Step 1: Load your JSON
with open("/Users/megz/Desktop/uni/MA - DTA/S2/Natural Language Processing/Shared task/filtered_pandora.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Step 2: Extract and flatten the authors list
flattened = []
for author in data["authors"]:
    row = {
        "id": author["id"],
        **author["labels"],  # spread labels into columns
        "comments": " ||| ".join(author["comments"])  # join comments with separator
    }
    flattened.append(row)

# Step 3: Create a DataFrame
df = pd.DataFrame(flattened)

# Step 4: Save to CSV
df.to_csv("authors_flat.csv", index=False)


In [41]:
len(df['comments'][2].split('.'))
df.head()

,id,agreeableness,conscientiousness,extraversion,neuroticism,openness,comments
0,-Areopagan-,0.00,0.96,0.60,0.01,0.99,Yeah I wouldnt want to deal with someone like ...
1,-BigSexy-,0.39,0.01,0.18,0.04,0.92,I'm not sure if it constitutes an official pol...
2,-BlitzN9ne,0.50,0.15,0.50,0.30,0.85,You're doing god's work ||| Man this is some q...
3,-CrestiaBell,0.50,0.50,0.85,0.50,0.85,Quick update: I got outbid on (potentially) an...
4,-dyad-,0.60,0.45,0.10,0.47,0.67,"Ok, this is adorable. ||| And he's completely..."


In [39]:
def train_personality_models(texts, trait_scores):
 
    print("Extracting embeddings from texts...")
    embeddings_list = []
    for text in texts:
        embeddings = get_embeddings(text)
        embeddings_list.append(embeddings)
    
 
    X = np.array(embeddings_list)

In [ ]:
import json
import torch
import numpy as np
import pandas as pd
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

# Load model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

# ========== EMBEDDING FUNCTIONS ==========

def embed_comment(comment_text):
    inputs = tokenizer(comment_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy()

def embed_long_comment(text, chunk_size=510, stride=128):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), stride):
        chunk = tokens[i:i+chunk_size]
        if len(chunk) == 0:
            continue
        chunk = tokenizer.build_inputs_with_special_tokens(chunk.tolist())
        chunk_inputs = torch.tensor([chunk])
        with torch.no_grad():
            outputs = model(chunk_inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        chunks.append(cls_embedding)
        if i + chunk_size >= len(tokens):
            break
    return np.mean(chunks, axis=0)

def get_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    return embed_comment(text) if len(tokens) <= 512 else embed_long_comment(text)

# ========== LOAD DATA ==========

with open("your_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# ========== PROCESS AUTHORS ==========

rows = []

for author in tqdm(data["authors"]):
    author_id = author["id"]
    labels = author["labels"]
    comments = author.get("comments", [])

    # Embed all comments, average them
    embeddings = []
    for comment in comments:
        try:
            vec = get_embedding(comment)
            embeddings.append(vec)
        except Exception as e:
            print(f"Error embedding comment for {author_id}: {e}")

    if embeddings:
        avg_embedding = np.mean(embeddings, axis=0)
        row = {
            "id": author_id,
            **labels
        }
        for i in range(len(avg_embedding)):
            row[f"embed_{i}"] = avg_embedding[i]
        rows.append(row)

# ========== SAVE TO CSV ==========

df = pd.DataFrame(rows)
df.to_csv("author_embeddings.csv", index=False)
print("Saved to author_embeddings.csv ✅")
